In [16]:
import copy
import os
import pandas as pd
import numpy as np
import json
import re
from pprint import pprint
from sklearn.model_selection import train_test_split as tts

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
DATA_DIR = "/content/drive/MyDrive/NLX/Final project/data-cleaning/qa-pairs"
OUT_DIR = "/content/drive/MyDrive/NLX/Final project/finetuning-data"

In [4]:
with open(os.path.join(DATA_DIR, 'Team-1-Q&A- JSON.txt'), 'r', encoding='utf-8') as infile:
    content = infile.read()
    json_blocks = re.findall(r'```(.*?)```', content, re.DOTALL)

    g1_temp = []

    for i, block in enumerate(json_blocks):

        block = block.replace('“', '"').replace('”', '"').replace("’", "'")
        pattern = re.compile(r'\{"input".*\n*", "output".*\}+')


        matches = pattern.findall(block)
        # print(type(matches), len(matches), type(matches[0]))
        for m in matches:
            try:
                m_temp = m.replace("\n", "")
                temp = json.loads(m_temp)
            except:
                # print(type(m), m_temp[-1], m)
                pass
            g1_temp.append(temp)

seen = set()
g1 = []
for d in g1_temp:
    # Use frozenset to handle dictionaries with non-primitive types
    marker = frozenset(d.items())
    if marker not in seen:
        seen.add(marker)
        g1.append(d)

print(type(g1), type(g1[0]))
print(len(g1))
g1[:3]

<class 'list'> <class 'dict'>
81


[{'input': ' What does the EU AI Act stand for?',
  'output': ' Proposal for a Regulation laying down harmonized rules for artificial intelligence.'},
 {'input': ' What is the primary purpose of the EU AI Act?',
  'output': ' The EU AI Act aims to regulate the sale and use of AI in the EU, ensuring the proper functioning of the single market by setting consistent standards across member states, and safeguarding health, safety, and fundamental rights.'},
 {'input': ' How does the EU AI Act categorize AI systems based on risk?',
  'output': ' The EU AI Act employs a risk-based approach, categorizing AI systems into four levels of risk: unacceptable, high, limited, and minimal/none, with specific regulations focusing on unacceptable and high-risk categories.'}]

In [5]:
with open(os.path.join(DATA_DIR, 'question-answer-pairs.jsonl'), 'r', encoding='utf-8') as infile:
    g2 = []
    for line in infile:
        g2.append(json.loads(line))

print(type(g2))
print(len(g2))
g2[0]

<class 'list'>
26


{'input': 'What are the challenges in AI regulation?',
 'output': 'Former Google CEO Eric Schmidt emphasized the challenge of defining what society wants to gain from AI and the need for a balance between regulating AI and investing in innovation.'}

In [6]:
with open(os.path.join(DATA_DIR, 'Team3_QA_Pairs.json'), 'r', encoding='utf-8') as infile:
    g3 = json.load(infile)

print(type(g3))
print(len(g3))
g3[0]

<class 'list'>
105


{'input': 'What were some of the controversial points discussed during the negotiations?',
 'output': 'Controversial points included generative AI and police use of face recognition surveillance.'}

In [7]:
with open(os.path.join(DATA_DIR, 'QAndA.json'), 'r', encoding='utf-8') as infile:
    g4 = json.load(infile)

print(type(g4))
print(len(g4))
g4[0]

<class 'list'>
111


{'input': " How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?",
 'output': 'China\'s AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI.'}

In [8]:
with open(os.path.join(DATA_DIR, 'QAPairs_team#5.json'), 'r', encoding='utf-8') as infile:
    g5_temp = json.load(infile)

g5 = [v for _, v in g5_temp.items()]

print(type(g5))
print(len(g5))
g5[0]

<class 'list'>
94


{'input': 'What are the twin goals of Singapore’s NAIS 2.0(National AI Strategy)?',
 'output': 'One is excellence: Singapore will selectively develop peaks of excellence in AI, to advance the field and maximize value creation. The other is empowerment: Singapore will raise up individuals, businesses, and communities to use AI with confidence, discernment, and trust.'}

In [9]:
dataset = []
for group_data in [g1, g2, g3, g4, g5]:
    dataset.extend(group_data)

len(dataset)

417

In [10]:
dataset[:3]

[{'input': ' What does the EU AI Act stand for?',
  'output': ' Proposal for a Regulation laying down harmonized rules for artificial intelligence.'},
 {'input': ' What is the primary purpose of the EU AI Act?',
  'output': ' The EU AI Act aims to regulate the sale and use of AI in the EU, ensuring the proper functioning of the single market by setting consistent standards across member states, and safeguarding health, safety, and fundamental rights.'},
 {'input': ' How does the EU AI Act categorize AI systems based on risk?',
  'output': ' The EU AI Act employs a risk-based approach, categorizing AI systems into four levels of risk: unacceptable, high, limited, and minimal/none, with specific regulations focusing on unacceptable and high-risk categories.'}]

In [11]:
with open(os.path.join(DATA_DIR, 'finetune-dataset.jsonl'), 'w') as outfile:
    for item in dataset:
        json_str = json.dumps(item)
        # json_str = json_str.replace("\u2019", "'")
        # json_str = json_str.replace("\u0092", "'")
        # json_str = json_str.replace("\u201c", "")
        outfile.write(json_str + '\n')

In [25]:
prompting_techniques = {
    "prompt_0": "",

    "prompt_1": """Give the most concise answers possible to questions about AI policy, considering you are an expert of AI policy. \n
    """,

    "prompt_2": """Give an answer to my query by modeling the following example:
My Query: Summarize the UK's legislation on AI.
Your Answer: The UK's legislation on AI is currently decentralized, with no specific comprehensive law governing AI. Instead, existing laws such as data protection legislation (e.g., the Data Protection Act 2018), equalities and privacy laws (e.g., the Equality Act 2010 and the Human Rights Act 1998), and intellectual property laws (e.g., the Copyright, Designs and Patents Act 1988) play a role in regulating various aspects of AI development and usage. These laws impact data collection, discrimination, human rights implications, intellectual property rights, and the limitations on AI decision-making and surveillance tools in the workplace.\n
    """,

    "prompt_3": """Give an answer to my query by modeling the following examples:
Summarize the UK's legislation on AI: The UK's legislation on AI is currently decentralized, with no specific comprehensive law governing AI. Instead, existing laws such as data protection legislation (e.g., the Data Protection Act 2018), equalities and privacy laws (e.g., the Equality Act 2010 and the Human Rights Act 1998), and intellectual property laws (e.g., the Copyright, Designs and Patents Act 1988) play a role in regulating various aspects of AI development and usage. These laws impact data collection, discrimination, human rights implications, intellectual property rights, and the limitations on AI decision-making and surveillance tools in the workplace.
What is the predicted impact of generative AI on jobs?: LinkedIn's analysis predicts that the jobs of 55% of the platform’s users will be impacted in some way by the adoption of generative AI.
What is the regulation around training powerful AI models in Europe?: Regulations in Europe mandate that developers must include drafting technical documentation, adhere to EU copyright laws, and provide detailed summaries of the content used for training. Moreover, for high-impact general purpose AI models that pose systemic risks, additional obligations apply. These obligations entail conducting model evaluations, assessing and mitigating systemic risks, performing adversarial testing, reporting serious incidents to the Commission, ensuring cybersecurity measures, and reporting on energy efficiency.
What are some drawbacks to the CASC approach?: Its rulemakings are inherently retroactive, it does not broadly ensure algorithmic rights for ADSs that do not qualify as CASC ADSs, and it does not resolve capacity issues at federal agencies.
How is AI related to the United States' geopolitical relations with China?: The U.S.-China relationship looms large over AI governance: as Beijing pursues a national strategy aimed at making China the global leader in “AI theories, technologies, and applications” by 2030, policymakers in Washington are struggling with how to place guardrails around AI development without undermining the United States’ technological edge.\n
    """,

    "prompt_4": """First, analyze the keywords in the query. Secondly, decipher the purpose of the query. Don’t explicitly write these. Your final answer should be a maximum of 3 sentences long. The first sentence should summarize what the question is asking. The second sentence should give the main answer to the query. The third sentence can be an additional point if you think some information is very important to the query. Format all the sentences into a single paragraph.

    """,
    "prompt_5": """Think of 3 possible different answers to the query but do not output them all. Only output the answer that is the shortest, most concrete, relevant to the query, and easy to understand by a college student. Do not include the reason for your pick.\n
    """
}

In [13]:
for k, v in prompting_techniques.items():
    print(k, v)

prompt_0 
prompt_1 
    Give the most concise answers possible to questions about AI policy, considering you are an expert of AI policy. 

    
prompt_2 
    Give an answer to my query by modeling the following example:
    My Query: Summarize the UK's legislation on AI.
    Your Answer: The UK's legislation on AI is currently decentralized, with no specific comprehensive law governing AI. Instead, existing laws such as data protection legislation (e.g., the Data Protection Act 2018), equalities and privacy laws (e.g., the Equality Act 2010 and the Human Rights Act 1998), and intellectual property laws (e.g., the Copyright, Designs and Patents Act 1988) play a role in regulating various aspects of AI development and usage. These laws impact data collection, discrimination, human rights implications, intellectual property rights, and the limitations on AI decision-making and surveillance tools in the workplace.

    
prompt_3 
    Give an answer to my query by modeling the following exa

In [14]:
# promptings = []
# questions = []
# answers = []
# train_texts = []
# test_texts = []

# with open(os.path.join(DATA_DIR, 'finetune-dataset.jsonl'), 'r') as infile:
#     for line in infile:

#         data = json.loads(line)

#         for prompt_technique, prompt_alteration in prompting_techniques.items():

#             q = f"{prompt_alteration}{data['input']}"
#             a = data['output']
#             train_text = f"Question:\n{q}\n\nAnswer:\n{a}"
#             test_text = f"Question:\n{q}\n\nAnswer:\n"

#             promptings.append(prompt_technique)
#             questions.append(q)
#             answers.append(a)
#             train_texts.append(train_text)
#             test_texts.append(test_text)


# df = pd.DataFrame(
#     {
#         'prompt_technique': promptings,
#         'question': questions,
#         'answer': answers,
#         'train_text': train_texts,
#         'test_text': test_texts
#     }
# )

# df.head()

In [28]:
questions = []
answers = []

with open(os.path.join(DATA_DIR, 'finetune-dataset.jsonl'), 'r') as infile:

    for line in infile:

        data = json.loads(line)
        q = data['input']
        a = data['output']

        questions.append(q)
        answers.append(a)

df_0 = pd.DataFrame(
    {
        'question': questions,
        'answer': answers,
    }
)

df_0.head()

,question,answer
0,What does the EU AI Act stand for?,Proposal for a Regulation laying down harmoni...
1,What is the primary purpose of the EU AI Act?,The EU AI Act aims to regulate the sale and u...
2,How does the EU AI Act categorize AI systems ...,"The EU AI Act employs a risk-based approach, ..."
3,What are the consequences for AI systems cate...,AI systems classified as posing an unacceptab...
4,What requirements must developers of high-ris...,Developers of high-risk AI systems must adher...


In [29]:
df_1 = copy.deepcopy(df_0)
df_1['question'] = prompting_techniques['prompt_1'] + df_1['question']
df_1.head()

,question,answer
0,Give the most concise answers possible to ques...,Proposal for a Regulation laying down harmoni...
1,Give the most concise answers possible to ques...,The EU AI Act aims to regulate the sale and u...
2,Give the most concise answers possible to ques...,"The EU AI Act employs a risk-based approach, ..."
3,Give the most concise answers possible to ques...,AI systems classified as posing an unacceptab...
4,Give the most concise answers possible to ques...,Developers of high-risk AI systems must adher...


In [30]:
df_2 = copy.deepcopy(df_0)
df_2['question'] = prompting_techniques['prompt_1'] + df_2['question']
df_2.head()

,question,answer
0,Give the most concise answers possible to ques...,Proposal for a Regulation laying down harmoni...
1,Give the most concise answers possible to ques...,The EU AI Act aims to regulate the sale and u...
2,Give the most concise answers possible to ques...,"The EU AI Act employs a risk-based approach, ..."
3,Give the most concise answers possible to ques...,AI systems classified as posing an unacceptab...
4,Give the most concise answers possible to ques...,Developers of high-risk AI systems must adher...


In [31]:
df_3 = copy.deepcopy(df_0)
df_3['question'] = prompting_techniques['prompt_1'] + df_3['question']
df_3.head()

,question,answer
0,Give the most concise answers possible to ques...,Proposal for a Regulation laying down harmoni...
1,Give the most concise answers possible to ques...,The EU AI Act aims to regulate the sale and u...
2,Give the most concise answers possible to ques...,"The EU AI Act employs a risk-based approach, ..."
3,Give the most concise answers possible to ques...,AI systems classified as posing an unacceptab...
4,Give the most concise answers possible to ques...,Developers of high-risk AI systems must adher...


In [32]:
df_4 = copy.deepcopy(df_0)
df_4['question'] = prompting_techniques['prompt_1'] + df_4['question']
df_4.head()

,question,answer
0,Give the most concise answers possible to ques...,Proposal for a Regulation laying down harmoni...
1,Give the most concise answers possible to ques...,The EU AI Act aims to regulate the sale and u...
2,Give the most concise answers possible to ques...,"The EU AI Act employs a risk-based approach, ..."
3,Give the most concise answers possible to ques...,AI systems classified as posing an unacceptab...
4,Give the most concise answers possible to ques...,Developers of high-risk AI systems must adher...


In [33]:
df_5 = copy.deepcopy(df_0)
df_5['question'] = prompting_techniques['prompt_1'] + df_5['question']
df_5.head()

,question,answer
0,Give the most concise answers possible to ques...,Proposal for a Regulation laying down harmoni...
1,Give the most concise answers possible to ques...,The EU AI Act aims to regulate the sale and u...
2,Give the most concise answers possible to ques...,"The EU AI Act employs a risk-based approach, ..."
3,Give the most concise answers possible to ques...,AI systems classified as posing an unacceptab...
4,Give the most concise answers possible to ques...,Developers of high-risk AI systems must adher...


In [36]:
df_0.to_csv(os.path.join(OUT_DIR, "finetune-dataset-0.csv"), index=False)
df_1.to_csv(os.path.join(OUT_DIR, "finetune-dataset-1.csv"), index=False)
df_2.to_csv(os.path.join(OUT_DIR, "finetune-dataset-2.csv"), index=False)
df_3.to_csv(os.path.join(OUT_DIR, "finetune-dataset-3.csv"), index=False)
df_4.to_csv(os.path.join(OUT_DIR, "finetune-dataset-4.csv"), index=False)
df_5.to_csv(os.path.join(OUT_DIR, "finetune-dataset-5.csv"), index=False)